In [1]:
import sys
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import pickle
import scipy.sparse as sp

# for datasets
env_path = "/home/cstansbu/miniconda3/envs/geneformer/lib/python3.10/site-packages/"
sys.path.insert(0, env_path)

import scanpy as sc
import anndata as an
from datasets import Dataset

In [5]:
dirpath = "/scratch/indikar_root/indikar1/cstansbu/HSC/geneformer_adata/"
file_list = glob.glob(f"{dirpath}*.anndata.h5ad")


def consolidate_cell_type(
    df: pd.DataFrame, celltype_columns: list, fill_value: str = 'iHSC'
) -> pd.DataFrame:
    """Consolidates cell type information from multiple columns into a single 'cell_type' column.

    Args:
        df: The DataFrame containing the cell type data.
        celltype_columns: List of column names to check for cell type values.
        fill_value (str, optional): The value to fill in 'cell_type' if none of the columns in `celltype_columns` exist in the DataFrame (default: 'iHSC').

    Returns:
        The DataFrame with a new 'cell_type' column containing consolidated values or `fill_value` if no overlap is found.
    """
    
    # Filter columns that exist in the DataFrame
    valid_columns = [col for col in celltype_columns if col in df.columns]

    if valid_columns:  
        # Find the first non-null value from the specified columns, row-wise
        df['cell_type'] = df[valid_columns].bfill(axis=1).iloc[:, 0]
    else:
        # If no valid columns, fill 'cell_type' with the provided fill_value
        df['cell_type'] = fill_value
    
    # Replace NaN with None to ensure the new column has the proper data type
    df['cell_type'] = df['cell_type'].astype('object')
    df['cell_type'] = df['cell_type'].where(pd.notnull(df['cell_type']), None)
    
    return df

celltype_columns = ['celltype', 'STD.CellType', 'dataset']

adatas = {}

for fpath in file_list:
    base_name = os.path.basename(fpath)
    file_id = base_name.replace(".anndata.h5ad", "")
    tmp = sc.read_h5ad(fpath)
    
    not_na = (tmp.var['ensemble_id'].notna()) 
    tmp = tmp[:, not_na].copy()
    
    break
    
    
#     tmp.var_names_make_unique()
    
#     tmp.obs = consolidate_cell_type(tmp.obs, celltype_columns)

#     print()
#     print(fpath)
#     print(tmp.obs.columns)
#     adatas[file_id] = tmp
    
# print(adatas.keys())

/home/cstansbu/miniconda3/envs/geneformer/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/cstansbu/miniconda3/envs/geneformer/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
adata = an.concat(adatas, 
                  label='dataset', 
                  index_unique="_", 
                  join="outer")
adata

In [ ]:
adata.obs['cell_type'].value_counts(dropna=False)